In [45]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Args:
    def __init__(self):
        self.use_valedges_as_input = True
        self.epochs = 150
        self.runs = 10
        self.dataset = "Cora"
        self.batch_size = 1152  
        self.testbs = 1152 
        self.maskinput = True
        self.mplayers = 1
        self.nnlayers = 3
        self.hiddim = 256
        self.ln = True
        self.lnnn = True
        self.res = False
        self.jk = True
        self.gnndp = 0.05
        self.xdp = 0.7
        self.tdp = 0.3
        self.gnnedp = 0.0
        self.predp = 0.05
        self.preedp = 0.4
        self.gnnlr = 0.0043
        self.prelr = 0.0024
        self.beta = 1
        self.alpha = 1.0
        self.use_xlin = True
        self.tailact = True
        self.twolayerlin = False
        self.increasealpha = False
        self.splitsize = -1
        self.probscale = 4.3
        self.proboffset = 2.8
        self.pt = 0.75
        self.learnpt = False
        self.trndeg = -1
        self.tstdeg = -1
        self.cndeg = -1
        self.predictor = 'cn1'
        self.depth = 1
        self.model = 'puregcn'
        self.save_gemb = False
        self.load = None
        self.loadmod = False
        self.savemod = False
        self.savex = False
        self.loadx = False
        self.cnprob = 0
        self.emb_dim=256
        self.K=8     #Hop you can modify

args = Args()

In [ ]:
from torch_geometric.nn import MessagePassing
from torch_sparse import SparseTensor
import torch as th


class NormalBasisConv(MessagePassing):
    def __init__(self, fixed=False, kwargs={}):
        super(NormalBasisConv, self).__init__()
        self.fixed = fixed
        if self.fixed:
            n_hidden = kwargs['n_hidden']
            self.register_buffer('three_term_relations', th.zeros(n_hidden, 3), persistent=False)
            self.fixed_relation_stored = False

    def message(self, x_j, norm):
        
        if isinstance(norm, SparseTensor):
            norm = norm.to_dense()
        return norm.unsqueeze(-1) * x_j

    def forward(self, edge_index,adj, last_h, second_last_h, inner_product_last=None, inner_product_second_last=None, is_training=True):
        '''if isinstance(adj, SparseTensor):
            row, col, norm = adj.coo()
            edge_index = th.stack([row, col], dim=0)
            if norm is None:
                norm = th.ones(row.size(0), device=row.device)
            else:
                norm = norm.to_dense()
        else:
            edge_index = (adj > 0).nonzero(as_tuple=False).t()
            norm = adj[edge_index[0], edge_index[1]]

        if edge_index.dim() != 2 or edge_index.size(0) != 2:
            raise ValueError("edge_index should have shape (2, num_edges)")'''


        rst=adj  




        '''orthogonalize'''
        if is_training:
            rst, inner_product_last, inner_product_second_last = self.orthogonalize(is_training,rst, last_h, second_last_h)
        else:
            rst, _, _ = self.orthogonalize(is_training,rst, last_h, second_last_h, inner_product_last, inner_product_second_last)

        th.cuda.empty_cache()
        return rst, inner_product_last, inner_product_second_last

    def orthogonalize(self,is_training, rst, last_h, second_last_h, inner_product_last=None, inner_product_second_last=None):
            """Orthogonalize `rst` wrt. `last_h` and `second_last_h`."""
            

            if isinstance(rst, SparseTensor):
                rst = rst.to_dense()
            if isinstance(last_h, SparseTensor):
                last_h = last_h.to_dense()
            if isinstance(second_last_h, SparseTensor):
                second_last_h = second_last_h.to_dense()
            

            if is_training:
                

                inner_product_last = th.matmul(rst, last_h.T)
                

                rst = rst - last_h* inner_product_last

            else:
                inner_product_last = th.zeros(1, device=rst.device)


            if is_training:
                

                inner_product_second_last = th.matmul(rst, second_last_h.T)

                
                rst = rst - second_last_h * inner_product_second_last
            else:
                inner_product_second_last = th.zeros(1, device=rst.device)

            rst = rst / th.clamp((th.norm(rst,dim=0)),1e-8)
            
            return rst, inner_product_last, inner_product_second_last


In [ ]:
import torch.nn as nn 
import torch
import torch.nn.functional as F
from torch_sparse import SparseTensor
from torch import Tensor
import torch_sparse
from typing import List, Tuple, Final


class PermIterator:
    def __init__(self, device, size, bs, training=True) -> None:
        self.bs = bs
        self.training = training
        self.idx = torch.randperm(size, device=device) if training else torch.arange(size, device=device)

    def __len__(self):
        return (self.idx.shape[0] + (self.bs - 1) * (not self.training)) // self.bs

    def __iter__(self):
        self.ptr = 0
        return self

    def __next__(self):
        if self.ptr + self.bs * self.training > self.idx.shape[0]:
            raise StopIteration
        ret = self.idx[self.ptr:self.ptr + self.bs]
        self.ptr += self.bs
        return ret

def generate_base_adj(data, split_edge, maskinput, batch_size):
    pos_train_edge = split_edge['train']['edge'].to(data.x.device).t()
    adjmask = torch.ones_like(pos_train_edge[0], dtype=torch.bool)


    for perm in PermIterator(adjmask.device, adjmask.shape[0], batch_size):
        if maskinput:
            adjmask[perm] = 0
            tei = pos_train_edge[:, adjmask]
            adj = SparseTensor.from_edge_index(
                tei, sparse_sizes=(data.num_nodes, data.num_nodes)
            ).to(pos_train_edge.device) 

            adjmask[perm] = 1
            adj = adj.to_symmetric()
        else:
            adj = data.adj_t

    return adj

def generate_adjs_for_k(data, split_edge, K, args, maskinput, batch_size):
    
    adj_list = []  

    if maskinput:
        
        pos_train_edge = split_edge['train']['edge'].to(data.x.device).t()
        
        for k in range(1, K + 1):
            adj_k_hop_list = []
            for perm in PermIterator(pos_train_edge.device, pos_train_edge.shape[1], batch_size):
                adjmask = torch.ones_like(pos_train_edge[0], dtype=torch.bool)
                adjmask[perm] = 0
                tei = pos_train_edge[:, adjmask]

                adj = SparseTensor.from_edge_index(
                    tei, sparse_sizes=(data.num_nodes, data.num_nodes)
                ).to(pos_train_edge.device)
                adj = adj.to_symmetric()
                
                #k-hop adj
                adj_k_hop = generate_adj_k_hop(adj, k)  
                adj_k_hop_list.append(adj_k_hop)
            
            adj_list.append(adj_k_hop_list)

    else:

        base_adj = data.adj_t  

        
        for k in range(1, K + 1):
            adj_k_hop = generate_adj_k_hop(base_adj, k)
            adj_list.append(adj_k_hop)

    return adj_list 



def generate_adj_0_1_hop(adj_1):
    
    device = adj_1.device()  

    

    try:
        loop_edge = torch.arange(adj_1.size(0), dtype=torch.int64, device=device)
        loop_edge = torch.stack([loop_edge, loop_edge], dim=0)  
        
    except TypeError as e:
        
        raise

    return SparseTensor.from_edge_index(loop_edge).to(device) 


def generate_adj_k_hop(adj, k):

    result_adj = adj
    for _ in range(k - 1):  
        result_adj = result_adj.matmul(adj)
        
    return result_adj

In [49]:
import torch
from torch import nn
from torch_sparse import SparseTensor
from torch import Tensor
import torch_sparse
from typing import List, Tuple, Final




def sparsesample(adj: SparseTensor, deg: int) -> SparseTensor:
    '''
    sampling elements from a adjacency matrix
    '''
    rowptr, col, _ = adj.csr()
    rowcount = adj.storage.rowcount()
    mask = rowcount > 0
    rowcount = rowcount[mask]
    rowptr = rowptr[:-1][mask]

    rand = torch.rand((rowcount.size(0), deg), device=col.device)
    rand.mul_(rowcount.to(rand.dtype).reshape(-1, 1))
    rand = rand.to(torch.long)
    rand.add_(rowptr.reshape(-1, 1))

    samplecol = col[rand]

    samplerow = torch.arange(adj.size(0), device=adj.device())[mask]

    ret = SparseTensor(row=samplerow.reshape(-1, 1).expand(-1, deg).flatten(),
                       col=samplecol.flatten(),
                       sparse_sizes=adj.sparse_sizes()).to_device(
                           adj.device()).coalesce().fill_value_(1.0)
    #print(ret.storage.value())
    return ret


def sparsesample2(adj: SparseTensor, deg: int) -> SparseTensor:
    '''
    another implementation for sampling elements from a adjacency matrix
    '''
    rowptr, col, _ = adj.csr()
    rowcount = adj.storage.rowcount()
    mask = rowcount > deg

    rowcount = rowcount[mask]
    rowptr = rowptr[:-1][mask]

    rand = torch.rand((rowcount.size(0), deg), device=col.device)
    rand.mul_(rowcount.to(rand.dtype).reshape(-1, 1))
    rand = rand.to(torch.long)
    rand.add_(rowptr.reshape(-1, 1))

    samplecol = col[rand].flatten()

    samplerow = torch.arange(adj.size(0), device=adj.device())[mask].reshape(
        -1, 1).expand(-1, deg).flatten()

    mask = torch.logical_not(mask)
    nosamplerow, nosamplecol = adj[mask].coo()[:2]
    nosamplerow = torch.arange(adj.size(0),
                               device=adj.device())[mask][nosamplerow]

    ret = SparseTensor(
        row=torch.cat((samplerow, nosamplerow)),
        col=torch.cat((samplecol, nosamplecol)),
        sparse_sizes=adj.sparse_sizes()).to_device(
            adj.device()).fill_value_(1.0).coalesce()  
    #assert (ret.sum(dim=-1) == torch.clip(adj.sum(dim=-1), 0, deg)).all()
    return ret


def sparsesample_reweight(adj: SparseTensor, deg: int) -> SparseTensor:
    '''
    another implementation for sampling elements from a adjacency matrix. It will also scale the sampled elements.
    
    '''
    rowptr, col, _ = adj.csr()
    rowcount = adj.storage.rowcount()
    mask = rowcount > deg

    rowcount = rowcount[mask]
    rowptr = rowptr[:-1][mask]

    rand = torch.rand((rowcount.size(0), deg), device=col.device)
    rand.mul_(rowcount.to(rand.dtype).reshape(-1, 1))
    rand = rand.to(torch.long)
    rand.add_(rowptr.reshape(-1, 1))

    samplecol = col[rand].flatten()

    samplerow = torch.arange(adj.size(0), device=adj.device())[mask].reshape(
        -1, 1).expand(-1, deg).flatten()
    samplevalue = (rowcount * (1/deg)).reshape(-1, 1).expand(-1, deg).flatten()

    mask = torch.logical_not(mask)
    nosamplerow, nosamplecol = adj[mask].coo()[:2]
    nosamplerow = torch.arange(adj.size(0),
                               device=adj.device())[mask][nosamplerow]

    ret = SparseTensor(row=torch.cat((samplerow, nosamplerow)),
                       col=torch.cat((samplecol, nosamplecol)),
                       value=torch.cat((samplevalue,
                                        torch.ones_like(nosamplerow))),
                       sparse_sizes=adj.sparse_sizes()).to_device(
                           adj.device()).coalesce()  
    #assert (ret.sum(dim=-1) == torch.clip(adj.sum(dim=-1), 0, deg)).all()
    return ret


def elem2spm(element: Tensor, sizes: List[int]) -> SparseTensor:
    # Convert adjacency matrix to a 1-d vector
    col = torch.bitwise_and(element, 0xffffffff)
    row = torch.bitwise_right_shift(element, 32)
    return SparseTensor(row=row, col=col, sparse_sizes=sizes).to_device(
        element.device).fill_value_(1.0)


def spm2elem(spm: SparseTensor) -> Tensor:
    # Convert 1-d vector to an adjacency matrix
    sizes = spm.sizes()
    elem = torch.bitwise_left_shift(spm.storage.row(),
                                    32).add_(spm.storage.col())
    #elem = spm.storage.row()*sizes[-1] + spm.storage.col()
    #assert torch.all(torch.diff(elem) > 0)
    return elem


def spmoverlap_(adj1: SparseTensor, adj2: SparseTensor) -> SparseTensor:
    '''
    Compute the overlap of neighbors (rows in adj). The returned matrix is similar to the hadamard product of adj1 and adj2
    '''
    assert adj1.sizes() == adj2.sizes()
    element1 = spm2elem(adj1)
    element2 = spm2elem(adj2)

    if element2.shape[0] > element1.shape[0]:
        element1, element2 = element2, element1

    idx = torch.searchsorted(element1[:-1], element2)
    mask = (element1[idx] == element2)
    retelem = element2[mask]
    '''
    nnz1 = adj1.nnz()
    element = torch.cat((adj1.storage.row(), adj2.storage.row()), dim=-1)
    element.bitwise_left_shift_(32)
    element[:nnz1] += adj1.storage.col()
    element[nnz1:] += adj2.storage.col()
    
    element = torch.sort(element, dim=-1)[0]
    mask = (element[1:] == element[:-1])
    retelem = element[:-1][mask]
    '''

    return elem2spm(retelem, adj1.sizes())


def spmnotoverlap_(adj1: SparseTensor,
                   adj2: SparseTensor) -> Tuple[SparseTensor, SparseTensor]:
    '''
    return elements in adj1 but not in adj2 and in adj2 but not adj1
    '''
    # assert adj1.sizes() == adj2.sizes()
    element1 = spm2elem(adj1)
    element2 = spm2elem(adj2)

    idx = torch.searchsorted(element1[:-1], element2)
    matchedmask = (element1[idx] == element2)

    maskelem1 = torch.ones_like(element1, dtype=torch.bool)
    maskelem1[idx[matchedmask]] = 0
    retelem1 = element1[maskelem1]

    retelem2 = element2[torch.logical_not(matchedmask)]
    return elem2spm(retelem1, adj1.sizes()), elem2spm(retelem2, adj2.sizes())


def spmoverlap_notoverlap_(
        adj1: SparseTensor,
        adj2: SparseTensor) -> Tuple[SparseTensor, SparseTensor, SparseTensor]:
    '''
    return elements in adj1 but not in adj2 and in adj2 but not adj1
    '''
    # assert adj1.sizes() == adj2.sizes()
    element1 = spm2elem(adj1)
    element2 = spm2elem(adj2)

    if element1.shape[0] == 0:
        retoverlap = element1
        retelem1 = element1
        retelem2 = element2
    else:
        idx = torch.searchsorted(element1[:-1], element2)
        matchedmask = (element1[idx] == element2)

        maskelem1 = torch.ones_like(element1, dtype=torch.bool)
        maskelem1[idx[matchedmask]] = 0
        retelem1 = element1[maskelem1]

        retoverlap = element2[matchedmask]
        retelem2 = element2[torch.logical_not(matchedmask)]
    sizes = adj1.sizes()
    return elem2spm(retoverlap,
                    sizes), elem2spm(retelem1,
                                     sizes), elem2spm(retelem2, sizes)


def adjoverlap(adj1: SparseTensor, 
               adj2: SparseTensor, 
               tarei: Tensor, 
               filled1: bool = False, 
               calresadj: bool = False, 
               cnsampledeg: int = -1, 
               ressampledeg: int = -1) -> SparseTensor:
    """
    A wrapper for functions above to compute adjacency overlaps.
    Ensure inputs are SparseTensors.
    """
    
    if not isinstance(adj1, SparseTensor):
        adj1 = SparseTensor.from_dense(adj1)
    if not isinstance(adj2, SparseTensor):
        adj2 = SparseTensor.from_dense(adj2)

   
    adj1 = adj1[tarei[0]]
    adj2 = adj2[tarei[1]]

    if calresadj:
        
        adjoverlap, adjres1, adjres2 = spmoverlap_notoverlap_(adj1, adj2)
        if cnsampledeg > 0:
            adjoverlap = sparsesample_reweight(adjoverlap, cnsampledeg)
        if ressampledeg > 0:
            adjres1 = sparsesample_reweight(adjres1, ressampledeg)
            adjres2 = sparsesample_reweight(adjres2, ressampledeg)
        return adjoverlap, adjres1, adjres2
    else:
      
        adjoverlap = spmoverlap_(adj1, adj2)
        if cnsampledeg > 0:
            adjoverlap = sparsesample_reweight(adjoverlap, cnsampledeg)

    return adjoverlap




class DropAdj(nn.Module):
    doscale: Final[bool] 

    def __init__(self, dp: float = 0.0, doscale=True) -> None:
        super().__init__()
        self.dp = dp
        self.register_buffer("ratio", torch.tensor(1 / (1 - dp)))
        self.doscale = doscale

    def forward(self, adj: SparseTensor) -> SparseTensor:
        if self.dp < 1e-6 or not self.training:
            return adj

       
        adj_dense = adj.to_dense()

      
        mask = torch.rand_like(adj_dense) > self.dp
        adj_dense = adj_dense * mask.float()  # mask

        
        if self.doscale:
            adj_dense = adj_dense * self.ratio

       
        adj_sparse = SparseTensor.from_dense(adj_dense)

        return adj_sparse

def sparse_diff(spm_x, spm_y):
    """
    Given 2 sparse tensor spm_x and spm_y, do the diff x - y.
    x: bs * nidx, y: bs * nidy
    require nidx >= nidy
    """
    return spmoverlap_notoverlap_(spm_x, spm_y)[1]


In [50]:

from torch_geometric.nn import GCNConv, SAGEConv
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import torch
#from utils import adjoverlap
from torch_sparse.matmul import spmm_max, spmm_mean, spmm_add
from torch_sparse import SparseTensor
import torch_sparse
from torch_scatter import scatter_add
from typing import Iterable, Final



class PureConv(nn.Module):
    aggr: Final[str]
    def __init__(self, indim, outdim, aggr="gcn") -> None:
        super().__init__()
        self.aggr = aggr
        if indim == outdim:
            self.lin = nn.Identity()
        else:
            raise NotImplementedError

    def forward(self, x, adj_t):
        x = self.lin(x)
        if self.aggr == "mean":
            return spmm_mean(adj_t, x)
        elif self.aggr == "max":
            return spmm_max(adj_t, x)[0]
        elif self.aggr == "sum":
            return spmm_add(adj_t, x)
        elif self.aggr == "gcn":
            norm = torch.rsqrt_((1+adj_t.sum(dim=-1))).reshape(-1, 1)
            x = norm * x
            x = spmm_add(adj_t, x) + x
            x = norm * x
            return x



convdict = {
    "gcn":
    GCNConv,
    "gcn_cached":
    lambda indim, outdim: GCNConv(indim, outdim, cached=True),
    "sage":
    lambda indim, outdim: GCNConv(
        indim, outdim, aggr="mean", normalize=False, add_self_loops=False),
    "gin":
    lambda indim, outdim: GCNConv(
        indim, outdim, aggr="sum", normalize=False, add_self_loops=False),
    "max":
    lambda indim, outdim: GCNConv(
        indim, outdim, aggr="max", normalize=False, add_self_loops=False),
    "puremax": 
    lambda indim, outdim: PureConv(indim, outdim, aggr="max"),
    "puresum": 
    lambda indim, outdim: PureConv(indim, outdim, aggr="sum"),
    "puremean": 
    lambda indim, outdim: PureConv(indim, outdim, aggr="mean"),
    "puregcn": 
    lambda indim, outdim: PureConv(indim, outdim, aggr="gcn"),
    "none":
    None
}

In [51]:
from torch_geometric.nn import GCNConv, SAGEConv
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import torch
from torch_sparse.matmul import spmm_max, spmm_mean, spmm_add
from torch_sparse import SparseTensor
import torch_sparse
from torch_scatter import scatter_add
from typing import Iterable, Final


class GCN(nn.Module):

    def __init__(self,
                 in_channels,
                 hidden_channels,
                 out_channels,
                 num_layers,
                 dropout,
                 ln=False,
                 res=False,
                 max_x=-1,
                 conv_fn="gcn",
                 jk=False,
                 edrop=0.0,
                 xdropout=0.0,
                 taildropout=0.0,
                 noinputlin=False):
        super().__init__()
        
        self.adjdrop = DropAdj(edrop)
        
        if max_x >= 0:
            tmp = nn.Embedding(max_x + 1, hidden_channels)
            nn.init.orthogonal_(tmp.weight)
            self.xemb = nn.Sequential(tmp, nn.Dropout(dropout))
            in_channels = hidden_channels
        else:
            self.xemb = nn.Sequential(nn.Dropout(xdropout)) #nn.Identity()
            if not noinputlin and ("pure" in conv_fn or num_layers==0):
                self.xemb.append(nn.Linear(in_channels, hidden_channels))
                self.xemb.append(nn.Dropout(dropout, inplace=True) if dropout > 1e-6 else nn.Identity())
        
        self.res = res
        self.jk = jk
        if jk:
            self.register_parameter("jkparams", nn.Parameter(torch.randn((num_layers,))))
            
        if num_layers == 0 or conv_fn =="none":
            self.jk = False
            return
        
        convfn = convdict[conv_fn]
        lnfn = lambda dim, ln: nn.LayerNorm(dim) if ln else nn.Identity()

        if num_layers == 1:
            hidden_channels = out_channels

        self.convs = nn.ModuleList()
        self.lins = nn.ModuleList()
        if "pure" in conv_fn:
            self.convs.append(convfn(hidden_channels, hidden_channels))
            for i in range(num_layers-1):
                self.lins.append(nn.Identity())
                self.convs.append(convfn(hidden_channels, hidden_channels))
            self.lins.append(nn.Dropout(taildropout, True))
        else:
            self.convs.append(convfn(in_channels, hidden_channels))
            self.lins.append(
                nn.Sequential(lnfn(hidden_channels, ln), nn.Dropout(dropout, True),
                            nn.ReLU(True)))
            for i in range(num_layers - 1):
                self.convs.append(
                    convfn(
                        hidden_channels,
                        hidden_channels if i == num_layers - 2 else out_channels))
                if i < num_layers - 2:
                    self.lins.append(
                        nn.Sequential(
                            lnfn(
                                hidden_channels if i == num_layers -
                                2 else out_channels, ln),
                            nn.Dropout(dropout, True), nn.ReLU(True)))
                else:
                    self.lins.append(nn.Identity())
        

    def forward(self, x, adj_t):
        x = self.xemb(x)
        jkx = []
        for i, conv in enumerate(self.convs):
            x1 = self.lins[i](conv(x, self.adjdrop(adj_t)))
            if self.res and x1.shape[-1] == x.shape[-1]: # residual connection
                x = x1 + x
            else:
                x = x1
            if self.jk:
                jkx.append(x)
        if self.jk: # JumpingKnowledge Connection
            jkx = torch.stack(jkx, dim=0)
            sftmax = self.jkparams.reshape(-1, 1, 1)
            x = torch.sum(jkx*sftmax, dim=0)
        return x

In [ ]:
import itertools
import torch as th
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_sparse import SparseTensor
from typing import Iterable
import numpy as np


class CNLinkPredictor(nn.Module):
    cndeg: int
    
    def __init__(self,
                 in_channels,
                 hidden_channels,
                 out_channels,
                 num_layers,
                 dropout,
                 edrop=0.0,
                 ln=False,
                 cndeg=-1,
                 use_xlin=False,
                 tailact=False,
                 twolayerlin=False,
                 beta=1.0):
        super().__init__()

        self.register_parameter("beta", nn.Parameter(beta * th.ones((1))))
        self.dropadj = DropAdj(edrop)
        lnfn = lambda dim, ln: nn.LayerNorm(dim) if ln else nn.Identity()

        self.xlin = nn.Sequential(nn.Linear(hidden_channels, hidden_channels),
                                  nn.Dropout(dropout, inplace=True), nn.ReLU(inplace=True),
                                  nn.Linear(hidden_channels, hidden_channels),
                                  lnfn(hidden_channels, ln), nn.Dropout(dropout, inplace=True), nn.ReLU(inplace=True)) if use_xlin else lambda x: 0

        self.xcnlin = nn.Sequential(nn.Linear(1, hidden_channels),
                                    nn.Dropout(dropout, inplace=True), nn.ReLU(inplace=True),
                                    nn.Linear(hidden_channels, hidden_channels),
                                    lnfn(hidden_channels, ln), nn.Dropout(dropout, inplace=True),
                                    nn.ReLU(inplace=True), nn.Linear(hidden_channels, hidden_channels) if not tailact else nn.Identity())
        
        self.xijlin = nn.Sequential(nn.Linear(in_channels, hidden_channels), lnfn(hidden_channels, ln),
                                    nn.Dropout(dropout, inplace=True), nn.ReLU(inplace=True),
                                    nn.Linear(hidden_channels, hidden_channels) if not tailact else nn.Identity())
        
        self.lin = nn.Sequential(nn.Linear(hidden_channels, hidden_channels),
                                 lnfn(hidden_channels, ln),
                                 nn.Dropout(dropout, inplace=True),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(hidden_channels, hidden_channels) if twolayerlin else nn.Identity(),
                                 lnfn(hidden_channels, ln) if twolayerlin else nn.Identity(),
                                 nn.Dropout(dropout, inplace=True) if twolayerlin else nn.Identity(),
                                 nn.ReLU(inplace=True) if twolayerlin else nn.Identity(),
                                 nn.Linear(hidden_channels, out_channels))
        

        self.mlp_sparse = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels), 
            nn.Dropout(dropout, inplace=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_channels, hidden_channels),  
            lnfn(hidden_channels, ln),  
            nn.Dropout(dropout, inplace=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_channels, hidden_channels) if not tailact else nn.Identity()
        )

  
        self.mlp_fusion = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.Dropout(dropout, inplace=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_channels, hidden_channels), 
            lnfn(hidden_channels, ln),
            nn.Dropout(dropout, inplace=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_channels, hidden_channels) if twolayerlin else nn.Identity(),
            lnfn(hidden_channels, ln) if twolayerlin else nn.Identity(),
            nn.Dropout(dropout, inplace=True) if twolayerlin else nn.Identity(),
            nn.ReLU(inplace=True) if twolayerlin else nn.Identity(),
            nn.Linear(hidden_channels, out_channels)  
        )


        self.cndeg = cndeg
        self.K = args.K
        self.convs = nn.ModuleList([NormalBasisConv() for _ in range(self.K)])
        self.ln=ln

        self.accumulated_inner_products = []
        self.batch_count = 0
        self.hidden_channels = hidden_channels
        self.current_batch_progress = 0 
        self.batch_size = None
        self.dynamic_xcnlin = None
        self.dropout=dropout
        self.tailact=tailact

    def initialize_xcnlin(self, adj_size):
        """Initialize self.xcnlin based on adjs[0].size(1)."""
        lnfn = lambda dim, ln: nn.LayerNorm(dim) if self.ln else nn.Identity()
        if self.dynamic_xcnlin is None:  
            in_channels = adj_size[1]
            lnfn = lambda dim: nn.LayerNorm(dim) if self.ln else nn.Identity()
            self.dynamic_xcnlin = nn.Sequential(
                nn.Linear(in_channels, self.hidden_channels),
                nn.Dropout(self.dropout, inplace=True),
                nn.ReLU(inplace=True),
                nn.Linear(self.hidden_channels, self.hidden_channels),
                lnfn(self.hidden_channels),
                nn.Dropout(self.dropout, inplace=True),
                nn.ReLU(inplace=True),
                nn.Linear(self.hidden_channels, self.hidden_channels) if not self.tailact else nn.Identity()
            ).to(device)

    def reset_constants(self, device, size_x):
        self.accumulated_inner_products = []


    def accumulate_constants(self, inner_product_sums, batch_size, device, batch_id):
        
        while len(self.accumulated_inner_products) <= batch_id:
            self.accumulated_inner_products.append([th.zeros_like(inner_product_sums[0][j], device=device) for j in range(2 * self.K)])

        for j in range(2 * self.K):

            if batch_id > 0:
                
                self.accumulated_inner_products[batch_id][j] = self.accumulated_inner_products[batch_id-1][j].to(device) + \
                    (1 / batch_id) * (inner_product_sums[batch_id][j] - self.accumulated_inner_products[batch_id-1][j])
                
            else:
               
                self.accumulated_inner_products[batch_id][j] = inner_product_sums[batch_id][j]




    def multidomainforward(self, x, adjs, tar_ei, filled1: bool = False, cndropprobs: Iterable[float] = [], batch_idx=None, is_training=True):
       
        device = x.device
        if self.batch_size is None:
            self.batch_size = tar_ei.size(1)

        if self.dynamic_xcnlin is None:
            self.initialize_xcnlin(adjs[0].size())

        if any(isinstance(adj, list) for adj in adjs):
            adjs = list(itertools.chain.from_iterable(adjs))

        xi = x[tar_ei[0]]
        xj = x[tar_ei[1]]
        xij = self.xijlin(xi * xj)

        x = x + self.xlin(x)

        
        cn_k=[]
        for k in range(args.K):
            cn_k.append(adjs[k])

        sparse_diffs=[]
        for k in range(args.K):
            sparse_diffs.append(cn_k[k])

       
        h0 = sparse_diffs[0] / th.clamp((th.norm(sparse_diffs[0],dim=0)), 1e-8)
    
        inner_product_sums = []
        if batch_idx is not None:
            while len(inner_product_sums) <= batch_idx:
                inner_product_sums.append([th.zeros((1, 1), device=device) for _ in range(2 * self.K)])
       
        second_last_h = th.zeros_like(h0, device=device)
        last_h = h0

        for i, (con, adj) in enumerate(zip(self.convs, sparse_diffs[1:]), 1):
            adj = adj.to(device)
            h_i = None  

            if is_training:
                h_i, inner_product_last, inner_product_second_last = con(tar_ei,adj, last_h, second_last_h, is_training=True)


                
                if batch_idx is not None:
                    inner_product_sums[batch_idx][2 * (i - 1)] += inner_product_last.to(device)
                    

                    inner_product_sums[batch_idx][2 * (i - 1) + 1] += inner_product_second_last.to(device)

            else:
                
                inner_product_last = self.accumulated_inner_products[2][2 * (i - 1)].to(device)


                inner_product_second_last = self.accumulated_inner_products[2][2 * (i - 1) + 1].to(device)


                h_i, _, _ = con(adj, last_h, second_last_h, inner_product_last, inner_product_second_last, is_training=False)

            if h_i is not None:
                second_last_h = last_h
                last_h = h_i.to(device)

                sparse_diffs[i] = h_i.to(device)


        if is_training:
            self.accumulate_constants(inner_product_sums, tar_ei.size(1), device, batch_idx)
          

      

        xs = []

        for xcn in sparse_diffs[k]:
            
            xcn_expanded = xcn.unsqueeze(-1).repeat(1, self.hidden_channels)  # (1, h) -> (h, hidden_channel)

            xcn_transformed = self.mlp_sparse(xcn_expanded.to(device))

            fused = xcn_transformed * self.beta + xij  

            fusion_transformed = self.mlp_fusion(fused.to(device))
 
            xs.append(fusion_transformed)


        xs = torch.cat(xs, dim=-1)

        return xs

    def forward(self, x, adj, tar_ei, filled1: bool = False, is_training=True):
        return self.multidomainforward(x, adj, tar_ei, filled1, [], None, is_training)


In [ ]:
predictor_dict = {

    "cn1": CNLinkPredictor

}

In [54]:
import torch
from sklearn.metrics import roc_auc_score, average_precision_score
from ogb.linkproppred import PygLinkPropPredDataset
import torch_geometric.transforms as T
from torch_sparse import SparseTensor
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import train_test_split_edges, negative_sampling, to_undirected
from torch_geometric.transforms import RandomLinkSplit




def randomsplit(dataset, val_ratio: float=0.10, test_ratio: float=0.2):
    def removerepeated(ei):
        ei = to_undirected(ei)
        ei = ei[:, ei[0]<ei[1]]
        return ei
    data = dataset[0]
    data.num_nodes = data.x.shape[0]
    data = train_test_split_edges(data, test_ratio, test_ratio)
    split_edge = {'train': {}, 'valid': {}, 'test': {}}
    num_val = int(data.val_pos_edge_index.shape[1] * val_ratio/test_ratio)
    data.val_pos_edge_index = data.val_pos_edge_index[:, torch.randperm(data.val_pos_edge_index.shape[1])]
    split_edge['train']['edge'] = removerepeated(torch.cat((data.train_pos_edge_index, data.val_pos_edge_index[:, :-num_val]), dim=-1)).t()
    split_edge['valid']['edge'] = removerepeated(data.val_pos_edge_index[:, -num_val:]).t()
    split_edge['valid']['edge_neg'] = removerepeated(data.val_neg_edge_index).t()
    split_edge['test']['edge'] = removerepeated(data.test_pos_edge_index).t()
    split_edge['test']['edge_neg'] = removerepeated(data.test_neg_edge_index).t()
    return split_edge

def loaddataset(name: str, use_valedges_as_input: bool, load=None):
    if name in ["Cora", "Citeseer", "Pubmed"]:
        dataset = Planetoid(root="dataset", name=name)
        split_edge = randomsplit(dataset)
        data = dataset[0]
        data.edge_index = to_undirected(split_edge["train"]["edge"].t())
        edge_index = data.edge_index
        data.num_nodes = data.x.shape[0]
    else:
        dataset = PygLinkPropPredDataset(name=f'ogbl-{name}')
        split_edge = dataset.get_edge_split()
        data = dataset[0]
        edge_index = data.edge_index
    data.edge_weight = None 
    print(data.num_nodes, edge_index.max())
    data.adj_t = SparseTensor.from_edge_index(edge_index, sparse_sizes=(data.num_nodes, data.num_nodes))
    data.adj_t = data.adj_t.to_symmetric().coalesce()
    data.max_x = -1
    if name == "ppa":
        data.x = torch.argmax(data.x, dim=-1)
        data.max_x = torch.max(data.x).item()
    elif name == "ddi":
        data.x = torch.arange(data.num_nodes)
        data.max_x = data.num_nodes
    if load is not None:
        data.x = torch.load(load, map_location="cpu")
        data.max_x = -1

    print("dataset split ")
    for key1 in split_edge:
        for key2  in split_edge[key1]:
            print(key1, key2, split_edge[key1][key2].shape[0])


    if use_valedges_as_input:
        val_edge_index = split_edge['valid']['edge'].t()
        full_edge_index = torch.cat([edge_index, val_edge_index], dim=-1)
        data.full_adj_t = SparseTensor.from_edge_index(full_edge_index, sparse_sizes=(data.num_nodes, data.num_nodes)).coalesce()
        data.full_adj_t = data.full_adj_t.to_symmetric()
    else:
        data.full_adj_t = data.adj_t
    return data, split_edge

In [ ]:
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_sparse import SparseTensor
import torch_geometric.transforms as T
from functools import partial
from sklearn.metrics import roc_auc_score, average_precision_score
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
from torch_geometric.utils import negative_sampling
from torch.utils.tensorboard import SummaryWriter

import time
from typing import Iterable

from torch_geometric.utils import k_hop_subgraph


def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)


from torch_sparse import SparseTensor
import torch
import torch.nn.functional as F

def CN_kij(adj: SparseTensor, k: int, ij: torch.Tensor, slice_size: int=512):
    device = adj.device() 
    ij = ij.to(device)  

    h = ij.shape[1]  
   
    n = adj.size(0)
    
    rets = []
    
   
    for idx in range(0, h, slice_size):
        tij = ij[:, idx: idx + slice_size]
        Ej = F.one_hot(tij[1].long(), num_classes=n).T.float().to(device) 
        
        Ei = F.one_hot(tij[0].long(), num_classes=n).float().to(device)   
       
        ret = []
       
        for _ in range(k): 
            Ej = adj @ Ej
            
            ret.append((Ei * Ej.T).sum(dim=-1))
           
        
        ret = torch.stack(ret, dim=-1)  # (slice_size, k)
      
        rets.append(ret)
        

  
    rets = torch.cat(rets, dim=0)
   

   
    adjs = [rets[:, i].unsqueeze(0) for i in range(k)]
   
    
    return adjs




def train(model, predictor, data, split_edge, optimizer, batch_size, maskinput=True, cnprobs=[], alpha=None):
    if alpha is not None:
        predictor.setalpha(alpha)

    model.train()
    predictor.train()
    pos_train_edge = split_edge['train']['edge'].to(data.x.device).t()
   

    total_loss = []
    negedge = negative_sampling(data.edge_index.to(pos_train_edge.device), data.adj_t.sizes()[0])
 
    

    for batch_idx, perm in enumerate(PermIterator(pos_train_edge.device, pos_train_edge.shape[1], batch_size)):
        optimizer.zero_grad()

        adjmask = torch.ones_like(pos_train_edge[0], dtype=torch.bool)
        if maskinput:
            adjmask[perm] = 0
            tei = pos_train_edge[:, adjmask]
            adj = SparseTensor.from_edge_index(tei, sparse_sizes=(data.num_nodes, data.num_nodes)).to(pos_train_edge.device)
            adjmask[perm] = 1
            adj = adj.to_symmetric()
           
        else:
            adj = data.adj_t

    
        h = model(data.x, adj)
        edge = pos_train_edge[:, perm]

        adjs = CN_kij(adj,args.K,edge)

        
        pos_outs = predictor.multidomainforward(h, adjs, edge, cndropprobs=cnprobs, batch_idx=batch_idx + 1)
        pos_loss = -F.logsigmoid(pos_outs).mean()
        
        edge = negedge[:, perm]
        neg_outs = predictor.multidomainforward(h, adjs, edge, cndropprobs=cnprobs, batch_idx=batch_idx + 1)
        neg_loss = -F.logsigmoid(-neg_outs).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        optimizer.step()
        print(f"[train] Loss: {loss.item()}")
        total_loss.append(loss.item())

       
        del adjmask, tei, adj, pos_outs, neg_outs, h, adjs
        torch.cuda.empty_cache()

    return np.mean(total_loss)


@torch.no_grad()
def test(model, predictor, data, split_edge, evaluator, batch_size,
         use_valedges_as_input):
    model.eval()
    predictor.eval()

    pos_train_edge = split_edge['train']['edge'].to(data.adj_t.device())
    pos_valid_edge = split_edge['valid']['edge'].to(data.adj_t.device())
    neg_valid_edge = split_edge['valid']['edge_neg'].to(data.adj_t.device())
    pos_test_edge = split_edge['test']['edge'].to(data.adj_t.device())
    neg_test_edge = split_edge['test']['edge_neg'].to(data.adj_t.device())

    adj = data.adj_t
    h = model(data.x, adj)

    
    pos_train_pred = torch.cat([
        predictor(h, CN_kij(adj,args.K,pos_train_edge[perm].t()), pos_train_edge[perm].t(), is_training=False).squeeze().cpu()
        for perm in PermIterator(pos_train_edge.device,
                                 pos_train_edge.shape[0], batch_size, False)
    ],
                               dim=0)


    pos_valid_pred = torch.cat([
        predictor(h, CN_kij(adj,args.K,pos_valid_edge[perm].t()), pos_valid_edge[perm].t(), is_training=False).squeeze().cpu()
        for perm in PermIterator(pos_valid_edge.device,
                                 pos_valid_edge.shape[0], batch_size, False)
    ],
                               dim=0)
    neg_valid_pred = torch.cat([
        predictor(h, CN_kij(adj,args.K,neg_valid_edge[perm].t()), neg_valid_edge[perm].t(), is_training=False).squeeze().cpu()
        for perm in PermIterator(neg_valid_edge.device,
                                 neg_valid_edge.shape[0], batch_size, False)
    ],
                               dim=0)
    if use_valedges_as_input:
        adj = data.full_adj_t
        h = model(data.x, adj)

    pos_test_pred = torch.cat([
        predictor(h, CN_kij(adj,args.K,pos_test_edge[perm].t()), pos_test_edge[perm].t(), is_training=False).squeeze().cpu()
        for perm in PermIterator(pos_test_edge.device, pos_test_edge.shape[0],
                                 batch_size, False)
    ],
                              dim=0)

    neg_test_pred = torch.cat([
        predictor(h,  CN_kij(adj,args.K,neg_test_edge[perm].t()), neg_test_edge[perm].t(),is_training=False).squeeze().cpu()
        for perm in PermIterator(neg_test_edge.device, neg_test_edge.shape[0],
                                 batch_size, False)
    ],
                              dim=0)

    results = {}
    for K in [20, 50, 100]:
        evaluator.K = K

        train_hits = evaluator.eval({
            'y_pred_pos': pos_train_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']

        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (train_hits, valid_hits, test_hits)
    return results, h.cpu()










def main():
    args = Args()
    print(args, flush=True)

    hpstr = str(args).replace(" ", "").replace("Namespace(", "").replace(
        ")", "").replace("True", "1").replace("False", "0").replace("=", "").replace("epochs", "").replace("runs", "").replace("save_gemb", "")
    writer = SummaryWriter(f"./rec/{args.model}_{args.predictor}")
    writer.add_text("hyperparams", hpstr)

    if args.dataset in ["Cora", "Citeseer", "Pubmed"]:
        evaluator = Evaluator(name=f'ogbl-ppa')
    else:
        evaluator = Evaluator(name=f'ogbl-{args.dataset}')

    device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
    data, split_edge = loaddataset(args.dataset, args.use_valedges_as_input, args.load)
    data = data.to(device)

    predfn = predictor_dict[args.predictor]
    if args.predictor != "cn0":
        predfn = partial(predfn, cndeg=args.cndeg)
    if args.predictor in ["cn1", "incn1cn1", "scn1", "catscn1", "sincn1cn1"]:
        predfn = partial(predfn, use_xlin=args.use_xlin, tailact=args.tailact, twolayerlin=args.twolayerlin, beta=args.beta)
    if args.predictor == "incn1cn1":
        predfn = partial(predfn, depth=args.depth, splitsize=args.splitsize, scale=args.probscale, offset=args.proboffset, trainresdeg=args.trndeg, testresdeg=args.tstdeg, pt=args.pt, learnablept=args.learnpt, alpha=args.alpha)
    
    ret = []

    for run in range(0, args.runs):
        set_seed(run)
        if args.dataset in ["Cora", "Citeseer", "Pubmed"]:
            data, split_edge = loaddataset(args.dataset, args.use_valedges_as_input, args.load) # get a new split of dataset
            data = data.to(device)
        bestscore = None
        
        
        model = GCN(data.num_features, args.hiddim, args.hiddim, args.mplayers,
                    args.gnndp, args.ln, args.res, data.max_x,
                    args.model, args.jk, args.gnnedp,  xdropout=args.xdp, taildropout=args.tdp, noinputlin=args.loadx).to(device)
        if args.loadx:
            with torch.no_grad():
                model.xemb[0].weight.copy_(torch.load(f"gemb/{args.dataset}_{args.model}_cn1_{args.hiddim}_{run}.pt", map_location="cpu"))
            model.xemb[0].weight.requires_grad_(False)
        predictor = predfn(args.hiddim, args.hiddim, 1, args.nnlayers,
                           args.predp, args.preedp, args.lnnn).to(device)
        if args.loadmod:
            keys = model.load_state_dict(torch.load(f"gmodel/{args.dataset}_{args.model}_cn1_{args.hiddim}_{run}.pt", map_location="cpu"), strict=False)
            print("unmatched params", keys, flush=True)
            keys = predictor.load_state_dict(torch.load(f"gmodel/{args.dataset}_{args.model}_cn1_{args.hiddim}_{run}.pre.pt", map_location="cpu"), strict=False)
            print("unmatched params", keys, flush=True)
        

        optimizer = torch.optim.Adam([{'params': model.parameters(), "lr": args.gnnlr}, 
           {'params': predictor.parameters(), 'lr': args.prelr}])
        
        for epoch in range(1, 1 + args.epochs):
            alpha = max(0, min((epoch-5)*0.1, 1)) if args.increasealpha else None
            t1 = time.time()
            loss = train(model, predictor, data, split_edge, optimizer,
                         args.batch_size, args.maskinput, [], alpha)
            print(f"trn time {time.time()-t1:.2f} s", flush=True)
            if True:
                t1 = time.time()
                results, h = test(model, predictor, data, split_edge, evaluator,
                               args.testbs, args.use_valedges_as_input)
                print(f"test time {time.time()-t1:.2f} s")
                if bestscore is None:
                    bestscore = {key: list(results[key]) for key in results}
                for key, result in results.items():
                    writer.add_scalars(f"{key}_{run}", {
                        "trn": result[0],
                        "val": result[1],
                        "tst": result[2]
                    }, epoch)

                if True:
                    for key, result in results.items():
                        train_hits, valid_hits, test_hits = result
                        if valid_hits > bestscore[key][1]:
                            bestscore[key] = list(result)
                            if args.save_gemb:
                                torch.save(h, f"gemb/{args.dataset}_{args.model}_{args.predictor}_{args.hiddim}.pt")
                            if args.savex:
                                torch.save(model.xemb[0].weight.detach(), f"gemb/{args.dataset}_{args.model}_{args.predictor}_{args.hiddim}_{run}.pt")
                            if args.savemod:
                                torch.save(model.state_dict(), f"gmodel/{args.dataset}_{args.model}_{args.predictor}_{args.hiddim}_{run}.pt")
                                torch.save(predictor.state_dict(), f"gmodel/{args.dataset}_{args.model}_{args.predictor}_{args.hiddim}_{run}.pre.pt")
                        print(key)
                        print(f'Run: {run + 1:02d}, '
                              f'Epoch: {epoch:02d}, '
                              f'Loss: {loss:.4f}, '
                              f'Train: {100 * train_hits:.2f}%, '
                              f'Valid: {100 * valid_hits:.2f}%, '
                              f'Test: {100 * test_hits:.2f}%')
                    print('---', flush=True)
        print(f"best {bestscore}")
        if args.dataset == "collab":
            ret.append(bestscore["Hits@50"][-2:])
        elif args.dataset == "ppa":
            ret.append(bestscore["Hits@100"][-2:])
        elif args.dataset == "ddi":
            ret.append(bestscore["Hits@20"][-2:])
        elif args.dataset == "citation2":
            ret.append(bestscore[-2:])
        elif args.dataset in ["Pubmed", "Cora", "Citeseer"]:
            ret.append(bestscore["Hits@100"][-2:])
        else:
            raise NotImplementedError
    ret = np.array(ret)
    print(ret)
    print(f"Final result: val {np.average(ret[:, 0]):.4f} {np.std(ret[:, 0]):.4f} tst {np.average(ret[:, 1]):.4f} {np.std(ret[:, 1]):.4f}")


if __name__ == "__main__":
    main()

2927963 tensor(2927962)
dataset split 
train source_node 30387995
train target_node 30387995
valid source_node 86596
valid target_node 86596
valid target_node_neg 86596
test source_node 86596
test target_node 86596
test target_node_neg 86596


KeyError: 'edge'